# Writing Azure and IoT code inside Polyglot Notebooks

Do you know you can sketch and test Azure operations and IoT code using Polyglot Notebooks? Let's see how to do that in this video.

In [1]:
<img src="https://github.com/marcoparenzan/AwesomePolyglotNotebooks/blob/main/notebooks/Writing%20Azure%20and%20IoT%20code%20inside%20Polyglot%20Notebooks/Slides/Slide1.PNG"/>

As a developer, I do most of my job inside Microsoft Visual Studio, mainly for product development or customer projects.
I use it also for sketching code during study sessions. The introduction in C# 9 of top level statements has simplified a lot my job without the need of writing boilerplate class code, when not really needed. But I cannot avoid the classic edit-compile-run-debug-edit cycle.
So here comes the dotnet Polyglot Notebooks where I can really sketch the code with fast and easy edit/run cycles.
Besides that, when I need to do operations, like creating Azure resources, I have the same need, sketching and trying Azure CLI or PowerShell commands, annotating parameters, failing and correcting commands more and more time.
Polyglot Notebooks can satisfy both these needs. Let's see how to do that in this video using Azure IoT as a scenario: creating the required Azure resources and writing some csharp code to send messages to the cloud.

Let's begin with operations in Azure and I will use Azure CLI inside PowerShell cells. I need to connect to my Azure subscription, so I can create Azure resources, mainly a Resource Group and an IoT Hub instance. Before doing that, two steps are necessary. First let's login to Azure. This will require to switch to the browser and login interactively at the Azure AD prompt.

In [ ]:
az login --output table

Then we need to ensure that you are using the correct subscription if you own or you have access to several subscriptions. The previous command will list all the available subs you can select, directly via string name.

In [ ]:
az account set --subscription "Visual Studio Ultimate with MSDN"

Now that you are logged in correctly, you can start creating the resources we need. We begin with the resource group and I assign it "awesomepolyglotnotebooks" as a name. I use westeurope as a location, but you can obviously choose the location you prefer and that it is useful for your business. The resource group is a logical resource to contain specific instances, but it is a good practice to locate the resource group in the same region the resources contained are deployed. So with AZ CLI I create the resource group.

In [ ]:
az group create --location westeurope --name awesomepolyglotnotebooks

Now I can create the Azure IoT Hub instance inside the resource group yet created. For convenience I assign the IoT Hub resource the same name as resource group, but you can obviously choose the name you prefer or need. As already told, the location is the same as the resource group, so westeurope again.

In [ ]:
az iot hub create --resource-group awesomepolyglotnotebooks --name awesomepolyglotnotebooks --location westeurope

Now I can create my first device, which I name it as testdevice, with shared secret as the authentication method, and added to the hub previously created.

In [ ]:
az iot hub device-identity create --device-id testdevice --auth-method shared_private_key --hub-name awesomepolyglotnotebooks

As a result, I have all the information about the new device. What are important here the primary and the secondary keys that allow the device to communicate with the IoT Hub instance. But with the keys is only a little unconfortable. A connection string is much more easy for most of the cases. So with the following command I can obtain it directly.

In [ ]:
az iot hub device-identity connection-string show  --device-id testdevice --hub-name awesomepolyglotnotebooks

All the Azure resources are ready: so we can start sketching some device code. Let's begin with the fundamental Nuget package to do the job: Microsoft.Azure.Devices.Client. So we can type the following code in a csharp shell to refer to the IoT class library.

In [ ]:
#r "nuget:Microsoft.Azure.Devices.Client"

The first time launching the ref command is always long to download and install, but after the time to finish decrease because the package will be already loaded from the local cache of the hosting pc. To refer to the classes inside the package, I need to specify the namespace:

In [ ]:
using Microsoft.Azure.Devices.Client;

Now I'm able to begin coding. In sequence, first I store the connection string in a variable.

In [ ]:
var connectionString = "";

Remember that the connection string to use is the one discovered before in the code.
Now I can include some namespaces, System.IO, System.Text.Json and System.Text to handle strings in the code and files:

In [ ]:
using System.IO;
using System.Text;
using System.Text.Json;

Now I can define the deviceClient object that is the proxy object to Azure IoT Hub. I can configure the object from a static method of the DeviceClient type that configure the object with a connection string:

In [ ]:
var deviceClient = DeviceClient.FromConnectionString(connectionString);

As I want to send some data, as in this demo I don't havusee a real device, I can use a sample log to read and generate the payloads. So let's begin writing some code to parse a csv file I have, the sampledata.csv file, and read the data using the System.IO package. What I want to do is reading the entire file as a string, then split the info among rows and columns. 

In [ ]:
var samples = File.ReadAllText("sampledata.csv").Split('\r').Select(xx => xx.Split(',').ToArray();

As there is an header row, but I know it, I skip the first row.

In [ ]:
samples = samples.Skip(1).ToArray();

What I know now, is that not using the timestamp written in the file, but reusing only the value measured, a temperature, I can select the 5th element on each line.

In [ ]:
var sample = samples[0];
sample[5]

So now I can compose the payload, serialize as Json, convert to a byte array. This is necessary because the Message object that comes from the Client library, require a payload expressed as a byte array to be sent to IoT Hub. So we can type the following code.

In [ ]:
var payload = new 
{
    Timestamp = DateTimeOffet.Now,
    Value = double.Parse(sample[5])
};
var payloadJson = JsonSerializer.Serialize(payload);
var payloadBytes = Encoding.UTF8.GetString(payloadJson);
var message = new Message(payloadBytes);

Now we have a message we can send. So we can invoke the SendEventAsync on the client object.

In [ ]:
await deviceClient.SendEventAsync(message);

For debugging purposes, and looking at some feedback from the code, we can log on the screen the json payload, for example. We can use the display object. Before foreach loop we can add the following line:

In [ ]:
var log = display("log goes here");

This code returns an object that represents the area under the cell it is executed. When updated, invoking the Update method, the new string message is replaced and visualized under the cell.

In [ ]:
log.Update(payloadJson);

And because the loop can be quite fast, we can slow down the code with a Delay instruction. Two seconds can be enough.

In [ ]:
await Task.Delay(2000);

So let's recap all the steps putting them inside a single loop over all the samples.

In [ ]:
var log = display("log goes here");
foreach(var sample in samples)
{
    var payload = new 
    {
        Timestamp = DateTimeOffet.Now,
        Value = double.Parse(sample[5])
    };
    var payloadJson = JsonSerializer.Serialize(payload);
    var payloadBytes = Encoding.UTF8.GetString(payloadJson);
    var message = new Message(payloadBytes);
    await deviceClient.SendEventAsync(message);
    log.Update(payloadJson);
    await Task.Delay(2000);
}

Great. So the code now is running and we receive the feedback with the payload that are sent. One thing is important to know. Until the code is running inside the loop, the cell is still executing, so we see that the cell is running and now other code can be executed. Consider this way of working as demo/testing code, as this notebook is and no production code. At any time, a running cell can be interrupted pressing on the square button, that syncronously stop the cell.

So we are reaching the end of the demo, as the events are sent. But one question. Can we check if the messages are correctly received? Yes we can, with the same advice we have just told. We can run some code that remain indefinetly waiting for events, so we have some blocking code inside a cell. For this reason, we can open a new notebook and run a command. We can use again AZ CLI for that. There is a command that can monitor events from a device. We can type the following code in a PowerShell cell in a new Notebook.

In [ ]:
az iot monitor-events --hub-name awesomepolyglotnotebooks --device-id testdevice

So we see that inside the running cell, the output shows that events are arriving. So this ends this demo. Remember to stop the cell so the command interrupts and you can continue to type code in cells.

Great. I hope you liked this session. The topic is quite easy as you probably has already seen the same code in many demos. What is innovative here is the entire code sketching experience inside a Polyglot Notebook. The takeaway of the session is the fact that you can move lot of you experimentation inside a Polyglot Notebook, because it is just .NET or commandline code. Obviously just to simplify with what you have seen here, but you can extend with all the possibilities, like multiple kernels, of .NET Interactive.

You can find the code of this session on my GitHub account repo, https://github.com/marcoparenzan/awesomepolyglotnotebooks. And see you in another video! Ciao!

In [1]:
<img src="https://github.com/marcoparenzan/AwesomePolyglotNotebooks/blob/main/notebooks/Writing%20Azure%20and%20IoT%20code%20inside%20Polyglot%20Notebooks/Slides/Slide2.PNG"/>